In [11]:
import cv2
import os
import numpy as np
import random
import noise

def create_binary_mask(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_silver = np.array([0, 0, 100])
    upper_silver = np.array([179, 100, 255])
    lower_very_light_dark = np.array([0, 0, 50])
    upper_very_light_dark = np.array([179, 50, 100])
    mask_silver = cv2.inRange(image_hsv, lower_silver, upper_silver)
    mask_very_light_dark = cv2.inRange(image_hsv, lower_very_light_dark, upper_very_light_dark)
    combined_mask = cv2.bitwise_or(mask_silver, mask_very_light_dark)
    kernel = np.ones((5, 5), np.uint8)
    binary_mask = cv2.dilate(combined_mask, kernel, iterations=1)
    return binary_mask

def create_cosine_image(width, height, max_amplitude=120, min_amplitude=70, periods=5, orientation='random'):
    if orientation == 'random':
        orientation = np.random.choice(['horizontal', 'vertical'])

    if orientation == 'horizontal':
        axis = np.linspace(0, periods * 2 * np.pi, width)
    elif orientation == 'vertical':
        axis = np.linspace(0, periods * 2 * np.pi, height)
    else:
        raise ValueError("Orientation must be either 'horizontal', 'vertical', or 'random'")

    amplitude_range = (max_amplitude - min_amplitude) / 2
    amplitude_mid = (max_amplitude + min_amplitude) / 2

    cosine_wave = amplitude_mid + amplitude_range * np.cos(axis)

    if orientation == 'horizontal':
        image = np.tile(cosine_wave, (height, 1))
    else:  # orientation == 'vertical'
        image = np.tile(cosine_wave[:, np.newaxis], (1, width))

    return image

def process_image_with_noise(image, noise_image, augmentation="yes"):
    binary_mask = create_binary_mask(image)
    if binary_mask is None or noise_image is None:
        print("Error: Binary mask or noise image is None.")
        return None

    noise_image = cv2.resize(noise_image, (binary_mask.shape[1], binary_mask.shape[0]))

    cos_wave_image = create_cosine_image(binary_mask.shape[1], binary_mask.shape[0], max_amplitude=255, min_amplitude=180)
    noise_image_modulated = (noise_image.astype(np.float32) * (cos_wave_image.astype(np.float32) / 255.0)).astype(np.uint8)

    mixed_image = cv2.bitwise_and(noise_image_modulated, binary_mask)
    if augmentation.lower() == "yes":
        mixed_image_resized = cv2.resize(mixed_image, (image.shape[1], image.shape[0]))
        mixed_image_rgb = cv2.cvtColor(mixed_image_resized, cv2.COLOR_GRAY2RGB)
        mask = mixed_image_resized > 0
        augmented_image = image.copy()
        augmented_image[mask] = mixed_image_rgb[mask]
    else:
        augmented_image = image.copy()

    return {
        'image': image,
        'anomaly_mask': binary_mask,
        'cos_wave_image': cos_wave_image,
        'noise_image': noise_image_modulated,
        'augmented_image': augmented_image
    }

def generate_wall_crack_noise_images(output_directory, num_images=557, image_size=(700, 700), max_crack_length=250, max_branches=0):
    noise_defect = []
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    for i in range(num_images):
        width, height = image_size
        crack_image = np.zeros((height, width), dtype=np.uint8)
        noise_scale = random.uniform(0.2, 0.4)
        step_size = random.uniform(2.0, 4.0)
        num_steps = random.randint(200, 400)
        num_cracks = random.randint(1, 3)
        for _ in range(num_cracks):
            start_pos = (random.randint(0, width), random.randint(0, height))
            perlin_curve_with_variable_thickness_crack(width, height, noise_scale, num_steps, step_size, crack_image, start_pos, max_crack_length, max_branches)
        angle = random.uniform(0, 360)
        M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
        crack_image = cv2.warpAffine(crack_image, M, (width, height), borderMode=cv2.BORDER_CONSTANT, borderValue=0)
        crack_image[crack_image > 0] = 255
        noise_defect.append(crack_image)
        filename = os.path.join(output_directory, f"crack_noise_{i:03d}.png")
        cv2.imwrite(filename, crack_image)
        print(f"Crack noise image saved: {filename}")
    return noise_defect

def generate_tooth_noise_image(image_size=(700, 700), num_strips_range=(1, 3), max_noise_size=50):
    width, height = image_size
    noise_defect = np.zeros((height, width), dtype=np.uint8)
    noise_scale = random.uniform(0.2, 0.4)
    step_size = random.uniform(2.0, 4.0)
    num_steps = random.randint(500, 800)
    num_strips = random.randint(*num_strips_range)
    for _ in range(num_strips):
        start_pos = (random.randint(0, width), random.randint(0, height))
        perlin_curve_with_variable_thickness_tooth(width, height, noise_scale, num_steps, step_size, noise_defect, start_pos, max_noise_size)
    angle = random.uniform(0, 360)
    M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
    noise_defect = cv2.warpAffine(noise_defect, M, (width, height))
    return noise_defect

def perlin_curve_with_variable_thickness_tooth(width, height, noise_scale, num_steps, step_size, noise_defect, start_pos, max_noise_size):
    x, y = start_pos
    last_x, last_y = x, y
    for i in range(num_steps):
        noise_value = noise.pnoise2(x * noise_scale, y * noise_scale, octaves=6)
        angle = noise_value * np.pi * 2
        x += np.cos(angle) * step_size
        y += np.sin(angle) * step_size
        if np.sqrt((x - start_pos[0])**2 + (y - start_pos[1])**2) > max_noise_size:
            break
        thickness = 3 + int(10 * abs(np.sin(i * np.pi / num_steps)))
        color = 255
        if 0 <= x < width and 0 <= y < height:
            cv2.line(noise_defect, (int(last_x), int(last_y)), (int(x), int(y)), color, thickness)
            last_x, last_y = x, y

def perlin_curve_with_variable_thickness_crack(width, height, noise_scale, num_steps, step_size, crack_image, start_pos, max_crack_length, max_branches):
    x, y = start_pos
    last_x, last_y = x, y
    total_length = 0
    for i in range(num_steps):
        noise_value = noise.pnoise2(x * noise_scale, y * noise_scale, octaves=6)
        angle = noise_value * np.pi * 2
        x += np.cos(angle) * step_size
        y += np.sin(angle) * step_size
        step_length = np.sqrt((x - last_x) ** 2 + (y - last_y) ** 2)
        total_length += step_length
        if total_length > max_crack_length:
            break
        if not (0 <= x < width and 0 <= y < height):
            break
        thickness = 1 + int(abs(np.sin(i * np.pi / num_steps) * np.cos(total_length / max_crack_length * np.pi)))
        thickness = min(thickness, 2)
        color = 255
        if random.random() < 0.9:
            cv2.line(crack_image, (int(last_x), int(last_y)), (int(x), int(y)), color, thickness)
        last_x, last_y = x, y

def create_branch(width, height, noise_scale, num_steps, step_size, crack_image, start_pos, max_crack_length, angle, thickness, remaining_branches):
    x, y = start_pos
    last_x, last_y = x, y
    total_length = 0
    for i in range(num_steps):
        noise_value = noise.pnoise2(x * noise_scale, y * noise_scale, octaves=6)
        angle = noise_value * np.pi * 2
        x += np.cos(angle) * step_size
        y += np.sin(angle) * step_size
        step_length = np.sqrt((x - last_x) ** 2 + (y - last_y) ** 2)
        total_length += step_length
        if total_length > max_crack_length:
            break
        if not (0 <= x < width and 0 <= y < height):
            break
        color = 255
        if random.random() < 0.9:
            cv2.line(crack_image, (int(last_x), int(last_y)), (int(x), int(y)), color, thickness)
        last_x, last_y = x, y

def augment_image_with_noise(image_path, noise_image, output_directory="augmented_images", noise_mask_directory="anomaly_masks", augmentation="yes"):
    image = cv2.imread(image_path)
    augmented_images = process_image_with_noise(image, noise_image, augmentation)
    if augmented_images is None:
        print(f"Error processing image: {image_path}")
        return

    base_filename = os.path.basename(image_path)
    base_name, ext = os.path.splitext(base_filename)

    augmented_image_filename = f"{base_name}_augmented{ext}"
    augmented_image_path = os.path.join(output_directory, augmented_image_filename)

    noise_image_filename = f"{base_name}_noise{ext}"
    noise_image_path = os.path.join(noise_mask_directory, noise_image_filename)

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    if not os.path.exists(noise_mask_directory):
        os.makedirs(noise_mask_directory)

    cv2.imwrite(augmented_image_path, augmented_images['augmented_image'])
    cv2.imwrite(noise_image_path, augmented_images['noise_image'])

    print(f"Augmented image saved: {augmented_image_path}")
    print(f"Noise image saved: {noise_image_path}")

def batch_augment_images(input_directory="input_images", output_directory="augmented_images", noise_mask_directory="anomaly_masks", augmentation="yes"):
    image_files = [f for f in os.listdir(input_directory) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if not image_files:
        print(f"No image files found in {input_directory}")
        return

    num_images = len(image_files)
    crack_noise_images = generate_wall_crack_noise_images(noise_mask_directory, num_images=num_images, image_size=(700, 700))
    tooth_noise_images = [generate_tooth_noise_image(image_size=(700, 700)) for _ in range(num_images)]

    for i, image_file in enumerate(image_files):
        image_path = os.path.join(input_directory, image_file)
        # Randomly choose between crack and tooth noise
        if random.choice([True, False]):
            noise_image = crack_noise_images[i % len(crack_noise_images)]
        else:
            noise_image = tooth_noise_images[i % len(tooth_noise_images)]
        augment_image_with_noise(image_path, noise_image, output_directory, noise_mask_directory, augmentation)

if __name__ == "__main__":
    input_directory = "/home/oumaima/Downloads/DREAM - version M2(1)/DREAM - version  M2/Version Finale M2 IMDS/data_set_1_sous_reseau/train/good_train"
    output_directory = "anomaly_image"
    noise_mask_directory = "noise_masks"

    batch_augment_images(input_directory=input_directory, 
                         output_directory=output_directory, 
                         noise_mask_directory=noise_mask_directory, 
                         augmentation="yes")


Crack noise image saved: noise_masks/crack_noise_000.png
Crack noise image saved: noise_masks/crack_noise_001.png
Crack noise image saved: noise_masks/crack_noise_002.png
Crack noise image saved: noise_masks/crack_noise_003.png
Crack noise image saved: noise_masks/crack_noise_004.png
Crack noise image saved: noise_masks/crack_noise_005.png
Crack noise image saved: noise_masks/crack_noise_006.png
Crack noise image saved: noise_masks/crack_noise_007.png
Crack noise image saved: noise_masks/crack_noise_008.png
Crack noise image saved: noise_masks/crack_noise_009.png
Crack noise image saved: noise_masks/crack_noise_010.png
Crack noise image saved: noise_masks/crack_noise_011.png
Crack noise image saved: noise_masks/crack_noise_012.png
Crack noise image saved: noise_masks/crack_noise_013.png
Crack noise image saved: noise_masks/crack_noise_014.png
Crack noise image saved: noise_masks/crack_noise_015.png
Crack noise image saved: noise_masks/crack_noise_016.png
Crack noise image saved: noise_